In [ ]:
import requests
import pandas as pd
import os
import urllib.request
from datetime import timedelta

### Lista de ativos

In [ ]:
caminho_dados='/home/brenno/Documentos/dados/base_dados_br_parquets'
chave_api = 'descubra'
headers = {'accept': 'application/json',
                'X-API-Key': chave_api}
os.chdir(caminho_dados)

In [ ]:
response = requests.get(f'https://api.fintz.com.br/bolsa/b3/avista/busca',  
                                        headers=headers)
df = pd.DataFrame(response.json())
df

In [ ]:
df[df['nome'].str.contains('RAIA')]

In [ ]:
cotacoes = pd.read_parquet("/home/brenno/Documentos/dados/base_dados_br_parquets/cotacoes.parquet")
cotacoes['data'] = pd.to_datetime(cotacoes['data']).dt.date
cotacoes['ticker'] = cotacoes['ticker'].astype(str)
cotacoes = cotacoes.sort_values('data', ascending=True)
cotacoes

In [ ]:
cotacoes_com_nomes = pd.merge(df, cotacoes, on = 'ticker')
cotacoes_com_nomes

In [ ]:
cotacoes_com_nomes[cotacoes_com_nomes['nome'].str.contains('SEQUOIA LOG')]

In [ ]:
ipos = pd.read_excel("/home/brenno/Documentos/dados/videos_youtube/2024/janeiro/IPOS.xlsx")
ipos = ipos[ipos['CLASSIFICAÇÃO'] == 'IPO']
ipos = ipos[ipos['INÍCIO DE NEGOCIAÇÃO'] > "2010-01-01"]
ipos = ipos[~ipos['LISTAGEM NA OFERTA'].str.contains('BDR')]
ipos = ipos[['NOME DE PREGÃO', 'INÍCIO DE NEGOCIAÇÃO', 'PREÇO POR AÇÃO / UNIT / BDR (LÇTO.)4']]
ipos.columns = ['nome', 'data', 'preco']
ipos['nome'] = ipos['nome'].str.replace("¹", "")
ipos['nome'] = ipos['nome'].str.replace("²", "")
ipos

In [ ]:
lista_ipos = ipos['nome'].to_list()

In [ ]:
cotacoes_com_nomes = cotacoes_com_nomes[cotacoes_com_nomes['nome'].isin(lista_ipos)]
cotacoes_com_nomes = cotacoes_com_nomes[cotacoes_com_nomes['nome'] != 'HRT PETROLEO']

In [ ]:
df_retornos_ipos = pd.DataFrame(columns = ['retorno_12m', 'retorno_24m', 'retorno_36m',
                                          'retorno_ibov_12m', 'retorno_ibov_24m', 'retorno_ibov_36m'], 
                                            index = lista_ipos)

In [ ]:
ibovespa = pd.read_parquet("ibov.parquet")
ibovespa['data'] = pd.to_datetime(ibovespa['data']).dt.date

In [ ]:
for empresa in lista_ipos:
    
    preco_de_ipo = ipos[ipos['nome'] == empresa]['preco'].iloc[0]
    cotacoes_empresa = cotacoes_com_nomes[cotacoes_com_nomes['nome'] == empresa]
    if cotacoes_empresa.empty == False:
        fator_de_ajuste = cotacoes_empresa['fator_ajuste'].iloc[0]
        preco_ipo_ajustado = preco_de_ipo * fator_de_ajuste
        data_inicial = cotacoes_empresa['data'].iloc[0]
        meses_12 = data_inicial + timedelta(days = 365)
        meses_24 = data_inicial + timedelta(days = 365 * 2)
        meses_36 = data_inicial + timedelta(days = 365 * 3)

        preco_12_meses = cotacoes_empresa[cotacoes_empresa['data'] < meses_12]['preco_fechamento_ajustado'].iloc[-1]
        preco_24_meses = cotacoes_empresa[cotacoes_empresa['data'] < meses_24]['preco_fechamento_ajustado'].iloc[-1]
        preco_36_meses = cotacoes_empresa[cotacoes_empresa['data'] < meses_36]['preco_fechamento_ajustado'].iloc[-1]

        retorno_12m = preco_12_meses/preco_ipo_ajustado - 1
        retorno_24m = preco_24_meses/preco_ipo_ajustado - 1
        retorno_36m = preco_36_meses/preco_ipo_ajustado - 1

        ibovespa_12m = ibovespa[(ibovespa['data'] >= data_inicial) & (ibovespa['data'] < meses_12)]
        ibovespa_24m = ibovespa[(ibovespa['data'] >= data_inicial) & (ibovespa['data'] < meses_24)]
        ibovespa_36m = ibovespa[(ibovespa['data'] >= data_inicial) & (ibovespa['data'] < meses_36)]

        retorno_ibov_12m = ibovespa_12m['fechamento'].iloc[-1]/ibovespa_12m['fechamento'].iloc[0] - 1
        retorno_ibov_24m = ibovespa_24m['fechamento'].iloc[-1]/ibovespa_24m['fechamento'].iloc[0] - 1
        retorno_ibov_36m = ibovespa_36m['fechamento'].iloc[-1]/ibovespa_36m['fechamento'].iloc[0] - 1


        df_retornos_ipos.loc[empresa] = [retorno_12m, retorno_24m, retorno_36m, 
                                         retorno_ibov_12m, retorno_ibov_24m, retorno_ibov_36m]  

In [ ]:
df_retornos_ipos.loc[df_retornos_ipos['retorno_24m'] == df_retornos_ipos['retorno_12m'], 'retorno_24m'] = pd.NA
df_retornos_ipos.loc[df_retornos_ipos['retorno_36m'] == df_retornos_ipos['retorno_24m'], 'retorno_36m'] = pd.NA
df_retornos_ipos.loc[df_retornos_ipos['retorno_36m'] == df_retornos_ipos['retorno_12m'], 'retorno_36m'] = pd.NA

In [ ]:
df_retornos_ipos

In [ ]:
df_retornos_ipos['maior_12'] = df_retornos_ipos['retorno_12m'] > df_retornos_ipos['retorno_ibov_12m']
df_retornos_ipos['maior_24'] = df_retornos_ipos['retorno_24m'] > df_retornos_ipos['retorno_ibov_24m']
df_retornos_ipos['maior_36'] = df_retornos_ipos['retorno_36m'] > df_retornos_ipos['retorno_ibov_36m']

df_retorno_ibov = df_retornos_ipos[['maior_12', 'maior_24', 'maior_36']]

In [ ]:
df_retornos_ipos.isna().sum()

# Análise de retornos 

In [ ]:
tabela_retornos_positivos = pd.DataFrame(columns = ['% da amostra'], index = ['12 meses', '24 meses', '36 meses'])
tabela_retornos_acima_do_ibov = pd.DataFrame(columns = ['% da amostra'], index = ['12 meses', '24 meses', '36 meses'])
empresas_com_maiores_retornos = pd.DataFrame(columns = ['Retorno', 'Empresa'], index = ['12 meses', '24 meses', '36 meses'])
empresas_com_menores_retornos = pd.DataFrame(columns = ['Retorno', 'Empresa'], index = ['12 meses', '24 meses', '36 meses'])
medias_e_medianas_de_retorno = pd.DataFrame(columns = ['Média', 'Mediana'], index = ['12 meses', '24 meses', '36 meses'])
composição_da_oferta = pd.DataFrame(columns = ['Varejo', 'Institucional', 'Estrangeiros', 'Outros'], 
                                   index = ['12M > 0', '12M < 0', '24M > 0', '24M < 0', '36M > 0', '36M < 0'])

In [ ]:
(df_retorno_ibov.sum()/len(df_retorno_ibov)).iloc[1]

In [ ]:
tabela_retornos_acima_do_ibov.iloc[0, 0] = str(round((df_retorno_ibov.sum()/len(df_retorno_ibov)).iloc[0] * 100, 2)) + "%"
tabela_retornos_acima_do_ibov.iloc[1, 0] = str(round((df_retorno_ibov.sum()/len(df_retorno_ibov)).iloc[1] * 100, 2)) + "%"
tabela_retornos_acima_do_ibov.iloc[2, 0] = str(round((df_retorno_ibov.sum()/len(df_retorno_ibov)).iloc[2] * 100, 2)) + "%"

In [ ]:
meses = [12, 24, 36]

for i, mes in enumerate(meses):

    df_retornos_ipos_xm = df_retornos_ipos[f'retorno_{mes}m']
    df_retornos_ipos_xm.dropna(inplace = True)
    df_retornos_ipos_xm = df_retornos_ipos_xm.astype(float)
    total_positivo = round((df_retornos_ipos_xm > 0).sum()/len(df_retornos_ipos_xm) * 100, 2) 
    
    tabela_retornos_positivos.iloc[i, 0] = str(total_positivo) + '%'
    
    retorno = round(df_retornos_ipos_xm.max() * 100, 2)
    empresa = df_retornos_ipos_xm.idxmax()
    
    empresas_com_maiores_retornos.iloc[i, 0] = retorno
    empresas_com_maiores_retornos.iloc[i, 1] = empresa

    retorno = round(df_retornos_ipos_xm.min() * 100, 2)
    empresa = df_retornos_ipos_xm.idxmin()
    
    empresas_com_menores_retornos.iloc[i, 0] = retorno
    empresas_com_menores_retornos.iloc[i, 1] = empresa

In [ ]:
medias_e_medianas_de_retorno.iloc[0, 0] = str(round((df_retornos_ipos.mean().iloc[0] * 100), 2)) + "%"
medias_e_medianas_de_retorno.iloc[0, 1] = str(round((df_retornos_ipos.median().iloc[0] * 100), 2)) + "%"
medias_e_medianas_de_retorno.iloc[1, 0] = str(round((df_retornos_ipos.mean().iloc[1] * 100), 2)) + "%"
medias_e_medianas_de_retorno.iloc[1, 1] = str(round((df_retornos_ipos.median().iloc[1] * 100), 2)) + "%"
medias_e_medianas_de_retorno.iloc[2, 0] = str(round((df_retornos_ipos.mean().iloc[2] * 100), 2)) + "%"
medias_e_medianas_de_retorno.iloc[2, 1] = str(round((df_retornos_ipos.median().iloc[2] * 100), 2)) + "%"

display(medias_e_medianas_de_retorno)

In [ ]:
ipos_2 = pd.read_excel("/home/brenno/Documentos/dados/videos_youtube/2024/janeiro/IPOS.xlsx")
ipos_2 = ipos_2[ipos_2['CLASSIFICAÇÃO'] == 'IPO']
ipos_2 = ipos_2[ipos_2['INÍCIO DE NEGOCIAÇÃO'] > "2010-01-01"]
ipos_2 = ipos_2[~ipos_2['LISTAGEM NA OFERTA'].str.contains('BDR')]
ipos_2

In [ ]:
ofertas_12m_maior_que_zero = df_retornos_ipos[df_retornos_ipos['retorno_12m'] > 0].index
comp_ofertas_12m_maior_que_zero = ipos_2[ipos_2['NOME DE PREGÃO'].isin(ofertas_12m_maior_que_zero)]
comp_ofertas_12m_maior_que_zero = comp_ofertas_12m_maior_que_zero[['VAREJO', 'INSTITUCIONAL', 
                                                                   'ESTRANGEIROS', 'OUTROS']]
composição_da_oferta.loc['12M > 0'] = [str(x) + "%" for x in round(comp_ofertas_12m_maior_que_zero.mean() * 100, 2).to_list()]

ofertas_12m_maior_que_zero = df_retornos_ipos[df_retornos_ipos['retorno_12m'] < 0].index
comp_ofertas_12m_maior_que_zero = ipos_2[ipos_2['NOME DE PREGÃO'].isin(ofertas_12m_maior_que_zero)]
comp_ofertas_12m_maior_que_zero = comp_ofertas_12m_maior_que_zero[['VAREJO', 'INSTITUCIONAL', 
                                                                   'ESTRANGEIROS', 'OUTROS']]
composição_da_oferta.loc['12M < 0'] = [str(x) + "%" for x in round(comp_ofertas_12m_maior_que_zero.mean() * 100, 2).to_list()]

ofertas_12m_maior_que_zero = df_retornos_ipos[df_retornos_ipos['retorno_24m'] > 0].index
comp_ofertas_12m_maior_que_zero = ipos_2[ipos_2['NOME DE PREGÃO'].isin(ofertas_12m_maior_que_zero)]
comp_ofertas_12m_maior_que_zero = comp_ofertas_12m_maior_que_zero[['VAREJO', 'INSTITUCIONAL', 
                                                                   'ESTRANGEIROS', 'OUTROS']]
composição_da_oferta.loc['24M > 0'] = [str(x) + "%" for x in round(comp_ofertas_12m_maior_que_zero.mean() * 100, 2).to_list()]

ofertas_12m_maior_que_zero = df_retornos_ipos[df_retornos_ipos['retorno_24m'] < 0].index
comp_ofertas_12m_maior_que_zero = ipos_2[ipos_2['NOME DE PREGÃO'].isin(ofertas_12m_maior_que_zero)]
comp_ofertas_12m_maior_que_zero = comp_ofertas_12m_maior_que_zero[['VAREJO', 'INSTITUCIONAL', 
                                                                   'ESTRANGEIROS', 'OUTROS']]
composição_da_oferta.loc['24M < 0'] = [str(x) + "%" for x in round(comp_ofertas_12m_maior_que_zero.mean() * 100, 2).to_list()]

ofertas_12m_maior_que_zero = df_retornos_ipos[df_retornos_ipos['retorno_36m'] > 0].index
comp_ofertas_12m_maior_que_zero = ipos_2[ipos_2['NOME DE PREGÃO'].isin(ofertas_12m_maior_que_zero)]
comp_ofertas_12m_maior_que_zero = comp_ofertas_12m_maior_que_zero[['VAREJO', 'INSTITUCIONAL', 
                                                                   'ESTRANGEIROS', 'OUTROS']]
composição_da_oferta.loc['36M > 0'] = [str(x) + "%" for x in round(comp_ofertas_12m_maior_que_zero.mean() * 100, 2).to_list()]

ofertas_12m_maior_que_zero = df_retornos_ipos[df_retornos_ipos['retorno_36m'] < 0].index
comp_ofertas_12m_maior_que_zero = ipos_2[ipos_2['NOME DE PREGÃO'].isin(ofertas_12m_maior_que_zero)]
comp_ofertas_12m_maior_que_zero = comp_ofertas_12m_maior_que_zero[['VAREJO', 'INSTITUCIONAL', 
                                                                   'ESTRANGEIROS', 'OUTROS']]
composição_da_oferta.loc['36M < 0'] = [str(x) + "%" for x in round(comp_ofertas_12m_maior_que_zero.mean() * 100, 2).to_list()]

composição_da_oferta

# Relatório de IPOs

### 126 empresas fizeram IPO desde 2010.

<br>
Das 126 empresas:

* 7 não apresentaram dados de 12 meses após IPO.
* 10 não apresentaram dados de 24 meses após IPO.
* 16 não apresentaram após de 36 meses após IPO.

# Fonte dos dados:

- Base de dados da Varos
- B3

### Investindo em evidências

## % das empresas com retorno positivo

In [199]:
tabela_retornos_positivos

,% da amostra
12 meses,47.06%
24 meses,40.52%
36 meses,40.0%


## % das empresas que superaram o ibovespa:

In [200]:
tabela_retornos_acima_do_ibov

,% da amostra
12 meses,44.44%
24 meses,37.3%
36 meses,33.33%


## Empresas com maiores e menores retornos

In [201]:
display(empresas_com_maiores_retornos)
empresas_com_menores_retornos

,Retorno,Empresa
12 meses,573.2,LOCAWEB
24 meses,274.95,IRBBRASIL RE
36 meses,413.1,INTERMEDICA


,Retorno,Empresa
12 meses,-84.6,MODALMAIS
24 meses,-93.19,INFRACOMM
36 meses,-95.89,SEQUOIA LOG


## Médias e medianas de retorno

In [202]:
medias_e_medianas_de_retorno

,Média,Mediana
12 meses,6.78%,-4.88%
24 meses,-2.08%,-23.86%
36 meses,-0.29%,-23.01%


## Composição das ofertas

In [203]:
composição_da_oferta

,Varejo,Institucional,Estrangeiros,Outros
12M > 0,7.95%,38.63%,47.26%,6.24%
12M < 0,10.14%,53.93%,30.86%,3.11%
24M > 0,7.81%,40.28%,45.85%,6.46%
24M < 0,9.54%,51.31%,33.8%,3.39%
36M > 0,8.06%,41.94%,42.79%,6.07%
36M < 0,8.87%,50.39%,36.72%,3.97%
